In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
def calculate_rsi(data, period=14):
    """
    Calculates the Relative Strength Index (RSI) for a given Pandas Series of closing prices.

    Parameters:
    data (pd.Series): A Pandas Series representing the closing prices of an asset.
    period (int, optional): The period over which to calculate the RSI. Default is 14.

    Returns:
    pd.Series: A Pandas Series containing the RSI values.
    """
    delta = data.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


def calculate_bollinger_bands_percent(data, period=20, sigma=2):
    """
    Calculates the Bollinger Bands Percentage (%B) for a given Pandas Series of closing prices.

    Parameters:
    data (pd.Series): A Pandas Series representing the closing prices of an asset.
    period (int, optional): The period over which to calculate the %B. Default is 20.

    Returns:
    pd.Series: A Pandas Series containing the %B values.
    """
    rolling_avg = data.rolling(period).mean()
    rolling_std = data.rolling(period).std()
    upper = rolling_avg + sigma * rolling_std
    lower = rolling_avg - sigma * rolling_std
    return (data - lower) / (upper - lower)


def calculate_stochastic_momentum_indicator(data, k_period=5, d_period=10, signal_period=5):
    """
    Calculates the Stochastic Momentum Indicator (SMI) for a given Pandas Series of closing prices.

    Parameters:
    data (pd.Series): A Pandas Series representing the closing prices of an asset.
    period (int, optional): The period over which to calculate the SMI. Default is 20.

    Returns:
    pd.Series: A Pandas Series containing the %B values.
    """
    # https://www.tradingview.com/support/solutions/43000707882-stochastic-momentum-index-smi/
    # https://www.amcharts.com/stochastic-momentum-index-indicator/
    delta = data.rolling(k_period).max() - data.rolling(k_period).min()
    relative = data - (data.rolling(k_period).max() + data.rolling(k_period).min()) / 2
    delta = delta.ewm(span=d_period, adjust=False).mean().ewm(span=d_period, adjust=False).mean()
    relative = relative.ewm(span=d_period, adjust=False).mean().ewm(span=d_period, adjust=False).mean()
    smi = 100 * (relative / delta)
    signal = smi.ewm(span=signal_period, adjust=False).mean()
    return smi, signal


In [3]:
#Ingests the report csv and identify the tickers that had 2767 (maximum datapoints)
#Does not use tickers with less than 2767 datapoints
rep_df = pd.read_csv("report.csv")
repuse_df= rep_df[rep_df.Datapoints == 2767]
print("Number of tickers to use in analysis: ", len(repuse_df))

Number of tickers to use in analysis:  383


In [4]:
#This code does several things
# 1. It gets all the tickers that has 2767 datapoints
# 2. It calculates the following indicates for each ticker (20 and 100 day EMA, MACD, RSI, CCI, BOLL, MA5/MA10, MTM6/MTM12, ROC, SMI)
# 3. In order to scale all ticker share prices to the same range, it takes the difference between the consecutive days (e.g., RSI(t) - RSI (t-1))
# 4. It takes the log of the differences
# 5. It normalizes the open, high, low, and close price by the previous day's close price and take the log value ensuring the normalized share prices for all tickers are in the same range for training stability
# 6. It normalizes the volume by the previous day's volume and take the log value
# 7. It calculates the labels (Change), which is whether the close price increases(1) or decreases(0) the next day
# 8. It converts everything to a numpy array and concatenate the array on each loop
# Final numpy array is shape (Ntickers, Ndays, Nfeatures)
# Ntickers = number of tickers = 383
# Ndays = number of days = 2767
# Nfeatures = number of features + 1 label
# Features+label are EMA20dif, EMA100dif, RSIdif, MACDdif, Opendif, Highdif, Lowdif, Closedif, Volumedif, Change
# 'dif' indicates the normarlize value (e.g., EMA20dif)
# Ignore runtime warning

try:
    del ticker_ary
except:
    pass

# get the tickers for the analysis
tickers = repuse_df.Ticker.values

#loop through all the tickers
for tick in tickers:
    #get the ticker file
    ticker_df = pd.read_csv(f'tick_data/{tick}.csv')
    ticker_temp2 = ticker_df.copy()

    #calculate the technical indicators
    indicators = ['EMA20','EMA100', 'RSI', 'MACD', 'CCI', 'BOLL', 'MA5/MA10', 'MTM6/MTM12', 'ROC', 'SMI']
    # EMAs and MACD
    ticker_temp2['EMA12'] = ticker_temp2['Close'].ewm(span=12, adjust=False).mean()
    ticker_temp2['EMA26'] = ticker_temp2['Close'].ewm(span=26, adjust=False).mean()
    ticker_temp2['EMA20'] = ticker_temp2['Close'].ewm(span=20, adjust=False).mean()
    ticker_temp2['EMA100'] = ticker_temp2['Close'].ewm(span=100, adjust=False).mean()
    ticker_temp2['MACD'] = ticker_temp2['EMA12'] - ticker_temp2['EMA26']
    # RSI
    ticker_temp2['RSI'] = calculate_rsi(ticker_temp2['Close'])
    # CCI
    ticker_temp2['p_t'] = (ticker_temp2['High'] + ticker_temp2['Low'] + ticker_temp2['Close']) / 3
    ticker_temp2['p_t_MAD'] = (ticker_temp2['p_t'] - ticker_temp2['p_t'].mean()).abs().mean()
    ticker_temp2['CCI'] = (ticker_temp2['p_t'] - ticker_temp2['p_t'].rolling(20).mean()) / (0.015 * ticker_temp2['p_t_MAD'])
    # ATR
    # ticker_temp2['TR'] = np.max(ticker_temp2['High'], ticker_temp2['Close'].shift(1)) - np.min(ticker_temp2['Low'], ticker_temp2['Close'].shift(1))
    ticker_temp2['Close_shift1'] = ticker_temp2['Close'].shift(1)
    ticker_temp2['TR'] = ticker_temp2[['High', 'Close_shift1']].max(axis=1) - ticker_temp2[['Low', 'Close_shift1']].min(axis=1)
    ticker_temp2['ATR'] = ticker_temp2['TR'].ewm(span=10, adjust=False).mean()
    # BOLL
    ticker_temp2['BOLL'] = calculate_bollinger_bands_percent(ticker_temp2['Close'])
    # MA5/MA10
    ticker_temp2['MA5'] = ticker_temp2['Close'].rolling(5).mean()
    ticker_temp2['MA10'] = ticker_temp2['Close'].rolling(10).mean()
    ticker_temp2['MA5/MA10'] = ticker_temp2['MA5'] / ticker_temp2['MA10']
    # MTM6/MTM12
    ticker_temp2['MTM6'] = ticker_temp2['Close'] / ticker_temp2['Close'].shift(6)
    ticker_temp2['MTM12'] = ticker_temp2['Close'] / ticker_temp2['Close'].shift(12)
    ticker_temp2['MTM6/MTM12'] = ticker_temp2['MTM6'] / ticker_temp2['MTM12']
    # ROC
    ticker_temp2['ROC'] = 100 * (ticker_temp2['Close'] - ticker_temp2['Close'].shift(10)) / ticker_temp2['Close'].shift(10)
    # SMI
    ticker_temp2['SMI'], ticker_temp2['SMI_signal'] = calculate_stochastic_momentum_indicator(ticker_temp2['Close'])

    # normalize the technical indicators
    # ticker_temp2[['EMA20dif','EMA100dif', 'RSIdif', 'MACDdif']] = ticker_temp2[['EMA20','EMA100', 'RSI', 'MACD']]/ticker_temp2[['EMA20','EMA100', 'RSI', 'MACD']].shift(1)
    # ticker_log = np.log(ticker_temp2[['EMA20dif','EMA100dif', 'RSIdif', 'MACDdif']])
    for i in indicators:
        ticker_temp2[i+'dif'] = ticker_temp2[i] / ticker_temp2[i].shift(1)
    ticker_log = np.log(ticker_temp2[[i+'dif' for i in indicators]])

    #normalize the share prices and volume (OHLCV)
    ticker_log['Opendif'] = np.log(ticker_temp2['Open']/ticker_temp2['Close'].shift(1))
    ticker_log['Highdif'] = np.log(ticker_temp2['High']/ticker_temp2['Close'].shift(1))
    ticker_log['Lowdif'] = np.log(ticker_temp2['Low']/ticker_temp2['Close'].shift(1))
    ticker_log['Closedif'] = np.log(ticker_temp2['Close']/ticker_temp2['Close'].shift(1))
    ticker_log['Volumedif']  = np.log(ticker_temp2['Volume']/ticker_temp2['Volume'].shift(1))

    #Calculate the label for each day
    ticker_log['Change'] = ((ticker_temp2['Close'].shift(-1)/ticker_temp2['Close']) > 1)*1
    ticker_log.dropna(inplace = True)

    #Convert to a numpy array
    ticker_temp = ticker_log.values

    #Add new ticker data to numpy array as we loop through the data
    try:
        ticker_ary  = np.concatenate((ticker_ary, ticker_temp[np.newaxis, :, :]), axis=0)
    except:
        ticker_ary = ticker_temp[np.newaxis, :, :]
    del ticker_temp


/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/christian/anaconda3/envs/a3/lib/python3.8/site-pack

In [5]:
#Example of data. Will need to do a batch normalization
ticker_log.head()

,EMA20dif,EMA100dif,RSIdif,MACDdif,CCIdif,BOLLdif,MA5/MA10dif,MTM6/MTM12dif,ROCdif,SMIdif,Opendif,Highdif,Lowdif,Closedif,Volumedif,Change
20,-0.003591,-0.001050,-0.438017,0.165135,0.428280,0.675740,-0.001618,-0.006660,0.378088,0.059490,-0.009171,-0.004248,-0.019757,-0.010159,0.120474,1
22,-0.003358,-0.001092,-0.103949,0.087259,-0.097826,1.258054,-0.004785,0.009139,-0.270137,0.052000,0.006237,0.012435,-0.007602,-0.004620,-0.585697,0
23,-0.003354,-0.001133,-0.018672,0.069712,0.098016,0.675917,-0.000203,-0.013320,0.255593,0.043418,-0.004310,0.006594,-0.009305,-0.003314,0.027348,0
24,-0.003290,-0.001162,-0.228402,0.053433,-0.006279,0.446387,0.001165,0.006612,0.242451,0.033936,-0.001328,0.003644,-0.004324,-0.002659,-0.121972,1
25,-0.000987,-0.000736,0.593685,-0.077754,-0.577483,1.356964,0.004492,0.009638,-0.846094,-0.052756,0.001995,0.024652,0.001995,0.021400,0.216201,1
